In [1]:
import bhnerf
from astropy import units
import jax
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import ruamel.yaml as yaml

# Runing on 2 GPUs
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1,3'

import warnings
from bhnerf.optimization import LogFn
warnings.simplefilter("ignore")

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-oi3yt11c because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to eht-imaging! v 1.2.2 



2023-03-17 13:03:59.233599: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs


# Load synthetic data
---

In [2]:
data_path = Path('../data/synthetic_lightcurves/flux_tube/sim1_lightcurve.csv')
lightcurves_df = pd.read_csv(data_path)
lightcurves_df.head()

,t,I,Q,U
0,9.340563,0.260572,0.096583,0.122080
1,9.350670,0.255594,0.102229,0.110132
2,9.360777,0.250520,0.106546,0.097898
3,9.370883,0.245289,0.109462,0.085561
4,9.380990,0.240443,0.110972,0.072903


# Recover 3D emission 
---
Recover the unknown 3D emission directly from the polarized lightcurves using [bh-NeRF](https://github.com/aviadlevis/bhnerf). \
Generate background accretion represented as a Gaussian Random Field (GRF) pre-computed using [pynoisy](https://github.com/aviadlevis/pynoisy)

In [3]:
with open(data_path.parent.joinpath('sim1_params.yaml'), 'r') as stream:
    simulation_params = yaml.load(stream, Loader=yaml.Loader)
locals().update(simulation_params['model'])

recovery_params = { 
    # Domain dimensions
    'z_width': 4,                                   # maximum disk width [M]
    'rmax': fov_M / 2,                              # maximum recovery radius
    'rmin': float(bhnerf.constants.isco_pro(spin)), # minimum recovery radius
    'recovery_scale': 1.0,                          # feature scale for recovery [M] 
    
    # Background accretion
    'grf_seed': 12,                                 # random seed of precomputed grf
    'alpha': 2.0,                                   # envelope  * exp(alpha * grf)
    'diameter_M': 15,                               # Gaussian envelope diameter (M)
    'H_r': 0.075,                                   # H/r (opening angle in the Z direction)
    'I_mean_disk': 2.2,                             # Jy
    'Q_frac_disk': 0.25,                            # Fraction of linear polarization
    
    # Optimization
    'stokes': ['Q', 'U'],
    'batchsize': 6,
    'sigma': 1,
    'hparams': {'num_iters': 50000, 'lr_init': 1e-4, 'lr_final': 1e-6, 'seed': 1}
}

locals().update(recovery_params)

In [70]:
# Load ground truth flare for comparison
sim_name = simulation_params['name']
flare_path = Path(simulation_params['flare_path'])
emission_flare = xr.load_dataarray(flare_path)
lightcurves_flare = np.array(lightcurves_df[stokes])
t_frames = np.array(lightcurves_df['t'])

# Compute geodesics
geos = bhnerf.kgeo.image_plane_geos(
    spin, np.deg2rad(inclination), 
    num_alpha=num_alpha, num_beta=num_beta, 
    alpha_range=[-fov_M/2, fov_M/2],
    beta_range=[-fov_M/2, fov_M/2])
t_injection = -float(geos.r_o + fov_M/4)

# Keplerian velocity and Doppler boosting
rot_sign = {'cw': -1, 'ccw': 1}
Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
g = bhnerf.kgeo.doppler_factor(geos, umu)

# Magnitude normalized magnetic field in fluid-frame
b = bhnerf.kgeo.magnetic_field_fluid_frame(geos, umu, **b_consts)
domain = np.bitwise_and(np.bitwise_and(np.abs(geos.z) < z_width, geos.r > rmin), geos.r < rmax)
b_mean = np.sqrt(np.sum(b[domain]**2, axis=-1)).mean()
b /= b_mean

# Polarized emission factors (including parallel transport)
J_inds = [['I', 'Q', 'U'].index(s) for s in stokes]
J = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac, V_frac=0), 0.0)[J_inds]
J_disk = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac_disk, V_frac=0), 0.0)

# Network / Optimization parameters
raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_start_obs*units.hr, J)
predictor = bhnerf.network.NeRF_Predictor(rmax, rmin, rmax, z_width, posenc_var=recovery_scale/fov_M)

""" Background accretion modeled as Gaussian Random Field (GRF) """
grf_path = '../data/synthetic_lightcurves/accretion_grfs/grf.seed{}.nc'.format(grf_seed)
grf = xr.load_dataarray(grf_path)
grf = grf.interp(t=float(grf.t[-1]) * (t_frames - t_frames.min()) / (t_frames.max() - t_frames.min()))
image_plane_disk = bhnerf.emission.grf_to_image_plane(grf, geos, Omega, J_disk, diameter_M, alpha, H_r)
lightcurves_disk = image_plane_disk.sum(axis=(-1,-2))
lightcurves_disk *=  I_mean_disk/lightcurves_disk.mean(axis=0)[0]
lightcurves_disk_qu = lightcurves_disk[:,J_inds]
disk_LP = np.sqrt(np.sum(lightcurves_disk_qu**2, axis=1)).mean()

# Update noisy target and sigmas for recovery
target = lightcurves_flare + lightcurves_disk_qu

In [13]:
%matplotlib widget
grf.visualization.animate()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
image_plane_xr = xr.DataArray(image_plane, {'t': t_frames, 'stokes': ['I', 'Q', 'U'], 'beta': geos.beta, 'alpha': geos.alpha}, ['t', 'stokes', 'beta', 'alpha'], 'Image Plane')
%matplotlib widget
fig, axes = plt.subplots(1, 3, figsize=(10, 3))
bhnerf.visualization.animate_movies_synced(image_plane_xr.transpose('stokes',...), axes, vmax=[0.01,None, None])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
%matplotlib widget
bhnerf.visualization.ipyvolume_3d(emission, fov_M)

interactive(children=(IntSlider(value=0, description='t', max=97), Output()), _dom_classes=('widget-interact',…

In [53]:
Q_frac_disk = 0.25
H_r=0.05
J_disk = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac_disk, V_frac=0), 0.0)


In [67]:
disk_width = 0.15
H_r = 0.075
image_plane_disk = bhnerf.emission.grf_to_image_plane(grf, geos, Omega, J_disk, alpha, disk_width, H_r)
lightcurves_disk = image_plane_disk.sum(axis=(-1,-2))
lightcurves_disk *=  I_mean_disk/lightcurves_disk.mean(axis=0)[0]
lightcurves_disk_qu = lightcurves_disk[:,J_inds]
disk_LP = np.sqrt(np.sum(lightcurves_disk_qu**2, axis=1)).mean()

In [115]:
diameter_M = 15.0
H_r = 0.05
grf = xr.load_dataarray(grf_path)
grf = grf.interp(t=float(grf.t[-1]) * (t_frames - t_frames.min()) / (t_frames.max() - t_frames.min()))
gaussian = bhnerf.utils.gaussian_xr([grf.y.size, grf.x.size], [0,0], std=diameter_M/2.355, fov=(fov_M, 'M')).data
movie = np.exp(alpha*grf) * gaussian
# Expand the 2D grf into 3D
emission = bhnerf.utils.expand_3d(movie, fov_z=fov_M, H_r=H_r)
emission.coords.update(bhnerf.utils.linspace_xr(emission.shape[1:], -fov_M/2, fov_M/2))
image_plane_disk = bhnerf.emission.image_plane_dynamics(emission, geos, Omega, 0.0, 0.0, J_disk, slow_light=False)

In [113]:
image_plane_xr = xr.DataArray(image_plane_disk, {'t': t_frames, 'stokes': ['I', 'Q', 'U'], 'beta': geos.beta, 'alpha': geos.alpha}, ['t', 'stokes', 'beta', 'alpha'], 'Image Plane')
%matplotlib widget
fig, axes = plt.subplots(1, 3, figsize=(10, 3))
bhnerf.visualization.animate_movies_synced(image_plane_xr.transpose('stokes',...), axes, vmax=[0.025, None, None])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
%matplotlib inline
log_period = 500
recovery_dir = data_path.parent.joinpath('recovery/{}.seed_{}.disk_LP_{:1.2}.grf_seed_{}.{}'.format(
    sim_name, hparams['seed'], disk_LP, grf_seed, ''.join(stokes)))
train_step = bhnerf.optimization.TrainStep.image(t_frames, target, sigma, dtype='lc')

writer = bhnerf.optimization.SummaryWriter(logdir=recovery_dir)
writer.add_images('emission/true', bhnerf.utils.intensity_to_nchw(emission_flare), global_step=0)
log_fns = [
    LogFn(lambda opt: writer.add_scalar('log_loss/train', np.log10(np.mean(opt.loss)), global_step=opt.step)), 
    LogFn(lambda opt: writer.recovery_3d(fov_M, emission_true=emission_flare)(opt), log_period=log_period),
    LogFn(lambda opt: writer.plot_lc_datafit(opt, target, stokes, t_frames, batchsize=20), log_period=log_period)
]

# Optimization
optimizer = bhnerf.optimization.Optimizer(hparams, predictor, raytracing_args, checkpoint_dir=recovery_dir)
optimizer.run(batchsize, train_step, raytracing_args, log_fns=log_fns)
writer.close()

params = {'simulation': simulation_params, 'recovery': recovery_params}
with open('{}/params.yaml'.format(recovery_dir), 'w') as file:
    yaml.dump(params, file, default_flow_style=False)

iteration:   0%|          | 0/50000 [00:00<?, ?it/s]